Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:
id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:
id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [1]:
import numpy as np
import pandas as pd

In [6]:
import csv
with open('checkins.dat') as input_file:        
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

In [7]:
with open('checkins.csv','w') as output_file:
    file_writer = csv.writer(output_file)
    file_writer.writerows(newLines)

In [9]:
data = pd.read_csv('checkins.csv', sep = ',', header = 0)

In [10]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [11]:
len(data)

396634

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах. 

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [12]:
from sklearn.cluster import MeanShift

In [19]:
data_full = data[['latitude', 'longitude']]
data_full.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [20]:
data_sample = data_full.sample(100000)

In [21]:
%%time
clst_sample = MeanShift(bandwidth = 0.1)
clst_sample.fit(data_sample)

Wall time: 4min 3s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [22]:
%%time
clst_full = MeanShift(bandwidth = 0.1)
clst_full.fit(data_upd)

Wall time: 37min 49s


MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [27]:
len(clst_sample.labels_)

100000

In [28]:
clst_sample.cluster_centers_[5]

array([  37.68662432, -122.40921976])

In [130]:
data_upd['cluster'] = clst_full.predict(data_upd)

C:\Users\chetv\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [131]:
data_upd

,latitude,longitude,cluster
0,38.895112,-77.036366,4
1,33.800745,-84.410520,7
2,45.523452,-122.676207,29
3,40.764462,-111.904565,92
4,33.448377,-112.074037,1
...,...,...,...
396629,40.850100,-73.866246,47
396630,33.748995,-84.387982,7
396631,42.765366,-71.467566,363
396632,42.439479,-83.743830,1056


In [132]:
cluster_size = pd.DataFrame(data_upd.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['cluster_size']

In [133]:
cluster_size

,cluster_size
cluster,
0,56187
1,10895
2,15282
3,9175
4,10942
...,...
5531,1
5532,1
5533,1


In [134]:
cluster_center_df = pd.DataFrame(clst_full.cluster_centers_)
cluster_center_df.columns = ['latitude', 'longitude']

In [135]:
cluster_center_df

,latitude,longitude
0,40.717485,-73.989359
1,33.449695,-112.002563
2,41.878165,-87.629827
3,33.446828,-111.902533
4,38.886107,-77.048199
...,...,...
5531,-38.451163,145.239041
5532,-41.218472,175.465151
5533,-41.867542,147.964198
5534,-43.365980,170.192856


In [136]:
cluster_df = cluster_size.join(cluster_center_df)

In [137]:
cluster_df

,cluster_size,latitude,longitude
cluster,,,
0,56187,40.717485,-73.989359
1,10895,33.449695,-112.002563
2,15282,41.878165,-87.629827
3,9175,33.446828,-111.902533
4,10942,38.886107,-77.048199
...,...,...,...
5531,1,-38.451163,145.239041
5532,1,-41.218472,175.465151
5533,1,-41.867542,147.964198


In [138]:
cluster_df = cluster_df[cluster_df.cluster_size > 15]

In [139]:
cluster_df

,cluster_size,latitude,longitude
cluster,,,
0,56187,40.717485,-73.989359
1,10895,33.449695,-112.002563
2,15282,41.878165,-87.629827
3,9175,33.446828,-111.902533
4,10942,38.886107,-77.048199
...,...,...,...
2335,17,41.530461,-93.955683
2665,22,37.783261,-121.542726
3588,20,42.097803,-88.530364


In [94]:
cluster_df.to_csv('clusters.csv', index = None)

In [95]:
office_coordinate = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

In [110]:
def distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5

In [111]:
def min_distance_to_office(lat, lot):
    min_dist = None
    for office_lat, office_lot in office_coordinate:
        dist = distance(lat, lot, office_lat, office_lot)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [140]:
cluster_df['min_distance'] = cluster_df.apply(lambda x: min_distance_to_office(x['latitude'], x['longitude']),axis=1)


C:\Users\chetv\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [141]:
cluster_df.sort_values('min_distance')[:20]

,cluster_size,latitude,longitude,min_distance
cluster,,,,
251,215,-33.866146,151.207082,0.003023
319,159,52.372489,4.892268,0.009625
317,165,25.846206,-80.311245,0.025084
55,1185,51.503055,-0.127113,0.051634
48,1258,33.811275,-118.144334,0.074644
24,2769,25.787086,-80.215128,0.135583
98,578,26.005052,-80.205598,0.181390
79,614,33.898488,-118.062259,0.194084
849,28,51.480366,-0.308323,0.233944


In [122]:
answer = cluster_df.sort_values('min_distance')[:1]

In [127]:
print(str(answer['latitude']), str(answer['longitude']))

cluster
203   -33.866087
Name: latitude, dtype: float64 cluster
203    151.208123
Name: longitude, dtype: float64


In [128]:
with open('answer.txt','w') as f:
    f.write(str(answer['latitude']))
    f.write(str(answer['longitude']))